In [1]:
import re
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
# 不用儲存article_id, 轉成list
def training_txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text))]
        for i in label:
            entity = i.split('\t')
            if int(entity[1]) > int(entity[2]):
                continue
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        for i, j in enumerate(text):
            if j == '，' or j == '。' or j == '？':
                label_list[i] = j
                
        text_list = re.split('\uff0c|\u3002|\uff1f', text)
        for sentence in text_list:
            x.append([i for i in sentence])
        x = x[:-1]
            
        sentence = list()
        for i in label_list:
            if i == '，' or i == '。' or i == '？':
                y.append(sentence)
                sentence = list()
            else:
                sentence.append(i)
                
    return [x, y]

In [3]:
x1, y1 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage1/SampleData_deid.txt')
x2, y2 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage2/train_1_update.txt')
x4, y4 = training_txt_to_list('/home/Danny/AI-CUP-2020/datasets/stage4/train_2.txt')

In [5]:
x = x1 + x2 + x4
y = y1 + y2 + y4
print(len(x))
print(len(y))

76162
76162


In [6]:
df = pd.DataFrame({'x': x, 'y': y})

In [7]:
df['x'] = df['x'].apply(lambda row : str(row))
df['y'] = df['y'].apply(lambda row : str(row))
df = df.drop_duplicates()
df['x'] = df['x'].apply(lambda row : eval(row))
df['y'] = df['y'].apply(lambda row : eval(row))
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
75713,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
75714,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
75716,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
75717,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [8]:
df = df.reset_index(drop=True)
df

,x,y
0,"[醫, 師, ：, 你, 有, 做, 超, 音, 波, 嘛]","[O, O, O, O, O, O, O, O, O, O]"
1,"[那, 我, 們, 來, 看, 報, 告]","[O, O, O, O, O, O, O]"
2,"[有, 些, 部, 分, 有, 紅, 字, 耶]","[O, O, O, O, O, O, O, O]"
3,"[民, 眾, ：, 紅, 字, 是, 甚, 麼, 意, 思]","[O, O, O, O, O, O, O, O, O, O]"
4,"[醫, 師, ：, 就, 是, 肝, 功, 能, 有, 比, 較, 高]","[O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...
31516,"[醫, 師, ：, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID]"
31517,"[醫, 師, ：, 6, 6, 6, 0, 8, 9, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."
31518,"[護, 理, 師, ：, 6, 6, 6, 0, 9, 8]","[O, O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID]"
31519,"[醫, 師, ：, 0, 9, 8, 7, 4, 0, 3]","[O, O, O, B-ID, I-ID, I-ID, I-ID, I-ID, I-ID, ..."


In [9]:
def find_contain_entity_rows(labels):
    dic_temp = {}
    dummy = 0
    for label in labels:
        dic_temp[label] = dummy
    #if len(dic_temp) > 1:
    #    return True
    if 'B-contact' in dic_temp:
        return True
    else:
        return False
df['flag'] = df['y'].apply(lambda l:find_contain_entity_rows(l))

In [24]:
df_entity = df[df['flag'] == True]

In [25]:
df_entity = df_entity.reset_index(drop=True)

In [26]:
df_entity = df_entity.drop(index=[28,30,35,38])
df_entity= df_entity.reset_index(drop=True)

In [27]:
df_entity

,x,y,flag
0,"[嗯, ⋯, ⋯, 就, 是, 臉, 書]","[O, O, O, O, O, B-contact, I-contact]",True
1,"[臉, 書]","[B-contact, I-contact]",True
2,"[民, 眾, ：, 臉, 書, 就, 是, ⋯, ⋯, 沒, 有]","[O, O, O, B-contact, I-contact, O, O, O, O, O, O]",True
3,"[臉, 書, 比, 較, 像, 是, ⋯, ⋯, 個, 管, 師, ：, 就, 是, 一, ...","[B-contact, I-contact, O, O, O, O, O, O, O, O,...",True
4,"[民, 眾, ：, G, r, i, n, d, r, 是, 哪, 一, 個, 啊]","[O, O, O, B-contact, I-contact, I-contact, I-c...",True
5,"[醫, 師, ：, H, o, r, n, e, t]","[O, O, O, B-contact, I-contact, I-contact, I-c...",True
6,"[G, r, i, n, d, r]","[B-contact, I-contact, I-contact, I-contact, I...",True
7,"[用, G, r, i, n, d, r]","[O, B-contact, I-contact, I-contact, I-contact...",True
8,"[我, 們, G, r, i, n, d, r, 有, 時, 候, 會, 有, 一, 些, ...","[O, O, B-contact, I-contact, I-contact, I-cont...",True
9,"[有, 時, 候, 因, 爲, 我, 們, 跟, G, r, i, n, d, r, 有, ...","[O, O, O, O, O, O, O, O, B-contact, I-contact,...",True


In [15]:
contact_df = pd.read_csv('../dictionary/online/contact.csv')

In [28]:
aug_df = pd.DataFrame([],columns=['x','y'])
aug_df

,x,y


In [29]:
for index, row in df_entity.iterrows():
    entity_start_pos = 0
    entity_end_pos = -1
    start_flag = False
    for i in range(len(row['y'])):
        if row['y'][i] == 'B-contact':
            entity_start_pos = i
            start_flag = True
        elif row['y'][i] == 'O' and start_flag == True:
            entity_end_pos = i
            break
        else:
            continue
    for entity in contact_df['entity_text']:
        sentence = []
        labels = []
        count = 0
        for i in range(entity_start_pos):
            sentence.append(row['x'][i])
            labels.append('O')
        sentence.append(entity[0])
        labels.append('B-contact')
        for i in range(1,len(entity)):
            sentence.append(entity[i])
            labels.append('I-contact')
        if entity_end_pos != -1:
            for i in range(entity_end_pos,len(row['x'])):
                sentence.append(row['x'][i])
                labels.append('O')
        #print(sentence)
        #print(labels)
        if len(sentence) != len(labels):
            print('Wrong')
        aug_df = aug_df.append({'x':sentence,'y':labels},ignore_index=True)

In [31]:
aug_df

,x,y
0,"[嗯, ⋯, ⋯, 就, 是, L, I, N, E]","[O, O, O, O, O, B-contact, I-contact, I-contac..."
1,"[嗯, ⋯, ⋯, 就, 是, 臉, 書]","[O, O, O, O, O, B-contact, I-contact]"
2,"[嗯, ⋯, ⋯, 就, 是, F, a, c, e, b, o, o, k]","[O, O, O, O, O, B-contact, I-contact, I-contac..."
3,"[嗯, ⋯, ⋯, 就, 是, G, r, i, n, d, r]","[O, O, O, O, O, B-contact, I-contact, I-contac..."
4,"[嗯, ⋯, ⋯, 就, 是, H, o, r, n, e, t]","[O, O, O, O, O, B-contact, I-contact, I-contac..."
...,...,...
547,"[用, F, B]","[O, B-contact, I-contact]"
548,"[用, f, b]","[O, B-contact, I-contact]"
549,"[用, t, w, i, t, t, e, r]","[O, B-contact, I-contact, I-contact, I-contact..."
550,"[用, T, w, i, t, t, e, r]","[O, B-contact, I-contact, I-contact, I-contact..."


In [32]:
aug_df = aug_df.sample(200)
aug_df = aug_df.reset_index(drop=True)
aug_df

,x,y
0,"[嗯, ⋯, ⋯, 就, 是, F, a, c, e, b, o, o, k]","[O, O, O, O, O, B-contact, I-contact, I-contac..."
1,"[民, 眾, ：, T, i, n, d, e, r, 是, 哪, 一, 個, 啊]","[O, O, O, B-contact, I-contact, I-contact, I-c..."
2,"[個, 管, 師, ：, 那, 如, 果, 還, 有, 其, 他, 的, 情, 況, 的, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[你, 有, 任, 何, 問, 題, 就, 可, 以, 在, f, b, 上, 詢, 問]","[O, O, O, O, O, O, O, O, O, O, B-contact, I-co..."
4,"[個, 管, 師, ：, 那, 如, 果, 還, 有, 其, 他, 的, 情, 況, 的, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
195,"[t, w, i, t, t, e, r, 跟, 那, 個]","[B-contact, I-contact, I-contact, I-contact, I..."
196,"[醫, 師, ：, 然, 後, 用, e, -, m, a, i, l]","[O, O, O, O, O, O, B-contact, I-contact, I-con..."
197,"[阿, 你, 在, f, b, 上, 面, 再, 告, 訴, 我]","[O, O, O, B-contact, I-contact, O, O, O, O, O, O]"
198,"[民, 眾, ：, G, r, i, n, d, r, 是, 藍, 色, 的]","[O, O, O, B-contact, I-contact, I-contact, I-c..."


In [33]:
aug_df.to_csv('../datasets/contact_augmentation.csv',index=False)